# Convert Locations in GeoJSON format

This notebook has the scope to convert the input dataset in a GeoJSON format for a better manipulation during the map view in the final application. 

## Import Libraries

In [6]:
import json
import pandas as pd 
import geopy
import geopy.distance
import sys

## Load Original Dataset

In [3]:
# import the dataset.
df = pd.read_csv('../../dtp/italy-dtp-2020-15.csv', header = 0, names=['time', 'lat', 'lon', 'discharge', 'temp', 'prec'])
df_drop = df.drop(columns = ['time', 'discharge', 'temp', 'prec'], inplace=False)
df_drop['coords'] = df_drop.apply(tuple, axis=1)
df_drop.head()

,lat,lon,coords
0,44.750378,7.56052,"(44.750378, 7.5605197)"
1,44.750378,7.56052,"(44.750378, 7.5605197)"
2,44.750378,7.56052,"(44.750378, 7.5605197)"
3,44.750378,7.56052,"(44.750378, 7.5605197)"
4,44.750378,7.56052,"(44.750378, 7.5605197)"


In [4]:
# retrieves unique locations.
ls = df_drop['coords'].unique()
print('Array: {}'.format(ls))

Array: [(44.750378, 7.5605197) (44.705326, 7.5624723) (44.66027, 7.564421) ...
 (41.324547, 13.706987) (41.2794, 13.704295) (41.23425, 13.701608)]


## Convert to GeoJSON

In the snippet below, we create a GeoJSON object with some additional features related to each point. These features are used to visualize additional information on the map usage.

### Generation of coordinates for rivers with a major grade of precision.

In [8]:
# loading of rivers,
with open('interesting.geojson', 'r') as fd:
    interest = json.load(fd)

# base object.
jsonObj = {
    "type": "FeatureCollection",
    "features": []
}

# GeoJSON building.
i = 0
added = []
for l in ls:
    ptCoord = geopy.Point(l[0], l[1])
    for key in interest.keys():
        coordList = interest[key]['geoms']['coordinates']
        for coord in coordList:
            couple = [coord[1], coord[0]]
            ptRiver = geopy.Point(coord[0], coord[1])
            dist = geopy.distance.distance(ptRiver, ptCoord).km
            if dist <= 2.6 and not couple in added:
                featureObj = {
                    "type": "Feature",
                    "name": key,
                    "id": i,
                    "geometry":{
                        "type":"Point",
                        "coordinates": couple, 
                        "ref": [l[1], l[0]]
                    }
                }
                i += 1
                jsonObj["features"].append(featureObj)
                added.append(couple)

# save results.
with open ('points_new_2.geojson', 'w') as fout:
    json.dump(jsonObj, fout)

### This one uses the measures point to create the map

In [5]:
# loading of rivers,
with open('interesting.geojson', 'r') as fd:
    interest = json.load(fd)

# base object.
jsonObj = {
    "type": "FeatureCollection",
    "features": []
}

# GeoJSON building.
i = 0
for l in ls:
    ptCoord = geopy.Point(l[0], l[1])
    for key in interest.keys():
        coordList = interest[key]['geoms']['coordinates']
        for coord in coordList:
            ptRiver = geopy.Point(coord[0], coord[1])
            dist = geopy.distance.distance(ptRiver, ptCoord).km
            if dist <= 2.6 :
                featureObj = {
                    "type": "Feature",
                    "name": key,
                    "id": i,
                    "geometry":{
                        "type":"Point",
                        "coordinates": [l[1], l[0]]
                    }
                }
                i += 1
                jsonObj["features"].append(featureObj)
                break

# save results.
with open ('points_new_fill.geojson', 'w') as fout:
    json.dump(jsonObj, fout)

### Generation on a GeoJson with only main points

In [8]:
# loading of rivers,
with open('interesting.geojson', 'r') as fd:
    interest = json.load(fd)

# base object.
jsonObj = {
    "type": "FeatureCollection",
    "features": []
}

# GeoJSON building.
i = 0

for l in ls:
    ptCoord = geopy.Point(l[0], l[1])
    minDist = sys.maxsize
    for key in interest.keys():
        coordList = interest[key]['geoms']['coordinates']
        for coord in coordList:
            couple = [coord[1], coord[0]]
            ptRiver = geopy.Point(coord[0], coord[1])
            dist = geopy.distance.distance(ptRiver, ptCoord).km
            if dist <= 2.6:
                if dist < minDist:
                    featureObj = {
                        "type": "Feature",
                        "name": key,
                        "id": i,
                        "geometry":{
                            "type":"Point",
                            "coordinates": couple, 
                            "ref": [l[1], l[0]]
                        }
                    }
                    minDist = dist
                else: 
                    i += 1
                    jsonObj["features"].append(featureObj)

# save results.
with open ('points_new_3.geojson', 'w') as fout:
    json.dump(jsonObj, fout)